In [1]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
import sklearn.preprocessing as preprocessing
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# from MulticoreTSNE import MulticoreTSNE as TSNE
import math
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances

# 1. import the real and the corresponding synthetic data

In [2]:
train_data = np.load('train_raw_correct.npy')
real_cor = np.corrcoef(np.transpose(train_data))

RUN_list = [1,2,3]

syn_data_emrwgan_list = []
syn_data_dpgan_list = []
syn_data_baseline_list = []
syn_data_medbgan_list = []
syn_data_medgan_list = []
syn_data_medwgan_list = []

for run in RUN_list:
    syn_data_emrwgan_list.append(np.transpose(np.load('./syn_emrwgan_'  + str(run) + '.npy')))
    syn_data_dpgan_list.append(np.transpose(np.load('./syn_dpgan_'  + str(run) + '.npy')))
    syn_data_baseline_list.append(np.transpose(np.load('./syn_baseline_'  + str(run) + '.npy')))
    syn_data_medbgan_list.append(np.transpose(np.load('./syn_medbgan_'  + str(run) + '.npy')))
    syn_data_medgan_list.append(np.transpose(np.load('./syn_medgan_'  + str(run) + '.npy')))
    syn_data_medwgan_list.append(np.transpose(np.load('./syn_medwgan_'  + str(run) + '.npy')))

# 2. compute results for latent cluster analysis

In [3]:
min_max_scaler = preprocessing.MinMaxScaler()
train_data_df = pd.DataFrame(train_data)
train_data_df.loc[:,[2588,2589,2590,2591,2592,2593,2594,2595]] = min_max_scaler.fit_transform(train_data_df[[2588,2589,2590,2591,2592,2593,2594,2595]].values)
NUM_C = 3 ## the number of clusters which has been optimized


In [4]:
## baseline
for matrix in syn_data_baseline_list:
    matrix_df = pd.DataFrame(np.transpose(matrix))
    matrix_df.loc[:,[2588,2589,2590,2591,2592,2593,2594,2595]] = min_max_scaler.fit_transform(matrix_df[[2588,2589,2590,2591,2592,2593,2594,2595]].values)
    mixed_data = np.concatenate((train_data_df.values,matrix_df.values), axis = 0)
    pca = PCA()
    pca_result = pca.fit_transform(mixed_data)
    sum_diag = np.sum(pca.explained_variance_ratio_)
    i = 1
    while  np.sum(pca.explained_variance_ratio_[:i]) < 0.8: # ensure the explained variance in the system is greater than 0.8
        i += 1
#     print(i, np.sum(pca.explained_variance_ratio_[:i])/np.sum(pca.explained_variance_ratio_))
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(mixed_data)
    
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    cluster_aff = kmeans_model.labels_.tolist()
    real_syn_label = [1]*len(train_data_df.values) + [0]*len(matrix_df.values)
    
    cluster_score_sum = 0
    for label in range(NUM_C):
        indices_label = [i for i in range(len(cluster_aff)) if cluster_aff[i] == label]
        real_syn_for_label = [real_syn_label[i] for i in indices_label]
        ratio = np.sum(real_syn_for_label)/len(real_syn_for_label)
        cluster_score_sum += (ratio - 0.5)**2
    log_cluster_score = math.log2(cluster_score_sum/NUM_C)
    print(log_cluster_score)

    

-2.485187662302252
-2.4865157687695096
-2.4841413085706368


In [5]:
## medgan
for matrix in syn_data_medgan_list:
    matrix_df = pd.DataFrame(np.transpose(matrix))
    matrix_df.loc[:,[2588,2589,2590,2591,2592,2593,2594,2595]] = min_max_scaler.fit_transform(matrix_df[[2588,2589,2590,2591,2592,2593,2594,2595]].values)
    mixed_data = np.concatenate((train_data_df.values,matrix_df.values), axis = 0)
    pca = PCA()
    pca_result = pca.fit_transform(mixed_data)
    sum_diag = np.sum(pca.explained_variance_ratio_)
    i = 1
    while  np.sum(pca.explained_variance_ratio_[:i]) < 0.8:
        i += 1
#     print(i, np.sum(pca.explained_variance_ratio_[:i])/np.sum(pca.explained_variance_ratio_))
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(mixed_data)
    
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    cluster_aff = kmeans_model.labels_.tolist()
    real_syn_label = [1]*len(train_data_df.values) + [0]*len(matrix_df.values)
    
    cluster_score_sum = 0
    for label in range(NUM_C):
        indices_label = [i for i in range(len(cluster_aff)) if cluster_aff[i] == label]
        real_syn_for_label = [real_syn_label[i] for i in indices_label]
        ratio = np.sum(real_syn_for_label)/len(real_syn_for_label)
        cluster_score_sum += (ratio - 0.5)**2
    log_cluster_score = math.log2(cluster_score_sum/NUM_C)
    print(log_cluster_score)

-2.5921804769296584
-2.531404379576881
-6.849002446050452


In [6]:
## medbgan
for matrix in syn_data_medbgan_list:
    matrix_df = pd.DataFrame(np.transpose(matrix))
    matrix_df.loc[:,[2588,2589,2590,2591,2592,2593,2594,2595]] = min_max_scaler.fit_transform(matrix_df[[2588,2589,2590,2591,2592,2593,2594,2595]].values)
    mixed_data = np.concatenate((train_data_df.values,matrix_df.values), axis = 0)
    pca = PCA()
    pca_result = pca.fit_transform(mixed_data)
    sum_diag = np.sum(pca.explained_variance_ratio_)
    i = 1
    while  np.sum(pca.explained_variance_ratio_[:i]) < 0.8:
        i += 1
#     print(i, np.sum(pca.explained_variance_ratio_[:i])/np.sum(pca.explained_variance_ratio_))
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(mixed_data)
    
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    cluster_aff = kmeans_model.labels_.tolist()
    real_syn_label = [1]*len(train_data_df.values) + [0]*len(matrix_df.values)
    
    cluster_score_sum = 0
    for label in range(NUM_C):
        indices_label = [i for i in range(len(cluster_aff)) if cluster_aff[i] == label]
        real_syn_for_label = [real_syn_label[i] for i in indices_label]
        ratio = np.sum(real_syn_for_label)/len(real_syn_for_label)
        cluster_score_sum += (ratio - 0.5)**2
    log_cluster_score = math.log2(cluster_score_sum/NUM_C)
    print(log_cluster_score)

-3.873172798339307
-12.43099074418064
-8.895436991793513


In [7]:
## emrwgan
for matrix in syn_data_emrwgan_list:
    matrix_df = pd.DataFrame(np.transpose(matrix))
    matrix_df.loc[:,[2588,2589,2590,2591,2592,2593,2594,2595]] = min_max_scaler.fit_transform(matrix_df[[2588,2589,2590,2591,2592,2593,2594,2595]].values)
    mixed_data = np.concatenate((train_data_df.values,matrix_df.values), axis = 0)
    pca = PCA()
    pca_result = pca.fit_transform(mixed_data)
    sum_diag = np.sum(pca.explained_variance_ratio_)
    i = 1
    while  np.sum(pca.explained_variance_ratio_[:i]) < 0.8:
        i += 1
#     print(i, np.sum(pca.explained_variance_ratio_[:i])/np.sum(pca.explained_variance_ratio_))
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(mixed_data)
    
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    cluster_aff = kmeans_model.labels_.tolist()
    real_syn_label = [1]*len(train_data_df.values) + [0]*len(matrix_df.values)
    
    cluster_score_sum = 0
    for label in range(NUM_C):
        indices_label = [i for i in range(len(cluster_aff)) if cluster_aff[i] == label]
        real_syn_for_label = [real_syn_label[i] for i in indices_label]
        ratio = np.sum(real_syn_for_label)/len(real_syn_for_label)
        cluster_score_sum += (ratio - 0.5)**2
    log_cluster_score = math.log2(cluster_score_sum/NUM_C)
    print(log_cluster_score)

-16.77139132419983
-12.142128349814652
-9.343728694930215


In [8]:
## medwgan
for matrix in syn_data_medwgan_list:
    matrix_df = pd.DataFrame(np.transpose(matrix))
    matrix_df.loc[:,[2588,2589,2590,2591,2592,2593,2594,2595]] = min_max_scaler.fit_transform(matrix_df[[2588,2589,2590,2591,2592,2593,2594,2595]].values)
    mixed_data = np.concatenate((train_data_df.values,matrix_df.values), axis = 0)
    pca = PCA()
    pca_result = pca.fit_transform(mixed_data)
    sum_diag = np.sum(pca.explained_variance_ratio_)
    i = 1
    while  np.sum(pca.explained_variance_ratio_[:i]) < 0.8:
        i += 1
#     print(i, np.sum(pca.explained_variance_ratio_[:i])/np.sum(pca.explained_variance_ratio_))
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(mixed_data)
    
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    cluster_aff = kmeans_model.labels_.tolist()
    real_syn_label = [1]*len(train_data_df.values) + [0]*len(matrix_df.values)
    
    cluster_score_sum = 0
    for label in range(NUM_C):
        indices_label = [i for i in range(len(cluster_aff)) if cluster_aff[i] == label]
        real_syn_for_label = [real_syn_label[i] for i in indices_label]
        ratio = np.sum(real_syn_for_label)/len(real_syn_for_label)
        cluster_score_sum += (ratio - 0.5)**2
    log_cluster_score = math.log2(cluster_score_sum/NUM_C)
    print(log_cluster_score)

-10.540151486272386
-10.0664187550477
-11.592408377772582


In [9]:
## dpgan
for matrix in syn_data_dpgan_list:
    matrix_df = pd.DataFrame(np.transpose(matrix))
    matrix_df.loc[:,[2588,2589,2590,2591,2592,2593,2594,2595]] = min_max_scaler.fit_transform(matrix_df[[2588,2589,2590,2591,2592,2593,2594,2595]].values)
    mixed_data = np.concatenate((train_data_df.values,matrix_df.values), axis = 0)
    pca = PCA()
    pca_result = pca.fit_transform(mixed_data)
    sum_diag = np.sum(pca.explained_variance_ratio_)
    i = 1
    while  np.sum(pca.explained_variance_ratio_[:i]) < 0.8:
        i += 1
#     print(i, np.sum(pca.explained_variance_ratio_[:i])/np.sum(pca.explained_variance_ratio_))
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(mixed_data)
    
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    cluster_aff = kmeans_model.labels_.tolist()
    real_syn_label = [1]*len(train_data_df.values) + [0]*len(matrix_df.values)
    
    cluster_score_sum = 0
    for label in range(NUM_C):
        indices_label = [i for i in range(len(cluster_aff)) if cluster_aff[i] == label]
        real_syn_for_label = [real_syn_label[i] for i in indices_label]
        ratio = np.sum(real_syn_for_label)/len(real_syn_for_label)
        cluster_score_sum += (ratio - 0.5)**2
    log_cluster_score = math.log2(cluster_score_sum/NUM_C)
    print(log_cluster_score)

-2.6811525107485115
-7.6239043090091485
-6.1323426427150425
